In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import operator
import re
import numpy as np
import random
import string
import collections, itertools
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools   import lru_cache, reduce
from statistics  import mean, median, mode, stdev, variance
from itertools   import (permutations, combinations, groupby, repeat,
                         islice, repeat,cycle, chain, product, zip_longest, takewhile, dropwhile, count as count_from)
from heapq       import heappush, heappop, heapify
from operator    import iand, ior, ilshift, irshift
from bisect      import bisect_left, bisect_right

In [2]:
import itertools

In [3]:
input_path = lambda day : "inputs/day{}.txt".format(day)
input_path_sample = lambda day : "inputs/day{}-sample.txt".format(day)

# Day 1: The Tyranny of the Rocket Equation

In [4]:
fuel = 0
with open(input_path(1), 'r') as f:
    for line in f:
        mass = int(line)
        fuel += mass // 3 - 2
print(fuel)

3160932


In [5]:
def compute_fuel(mass):
    ans = 0
    while True:
        mass = mass // 3 - 2
        if mass <= 0:
            break
        ans += mass
    return ans
assert(compute_fuel(100756) == 50346)
assert(compute_fuel(14) == 2)
assert(compute_fuel(1969) == 966)
fuel = 0
with open(input_path(1), 'r') as f:
    for line in f:
        mass = int(line)
        fuel += compute_fuel(mass)
print(fuel)

4738549


# Day 2: 1202 Program Alarm

In [6]:
def parse_code(code):
    import operator
    n = len(code)
    for i in range(0, n, 4):
        opcode = code[i]
        if opcode == 99:
            break
        else:
            left_idx = code[i + 1]
            right_idx = code[i + 2]
            r_idx = code[i + 3]
            op = operator.add if opcode == 1 else operator.mul
            code[r_idx] = op(code[left_idx], code[right_idx])
    return code
assert(parse_code([1,0,0,0,99]) == [2,0,0,0,99])
assert(parse_code([2,3,0,3,99]) == [2,3,0,6,99])
assert(parse_code([2,4,4,5,99,0]) == [2,4,4,5,99,9801])
assert(parse_code([1,1,1,4,99,5,6,0,99]) == [30,1,1,4,2,5,6,0,99])

In [7]:
with open(input_path(2), 'r') as f:
    memory = list(map(int, f.read().split(',')))
code = memory[:]
code[1] = 12
code[2] = 2
print(parse_code(code)[0])

2890696


In [8]:
def choose_noun_and_verb(target, memory):
    n = len(memory)
    for noun in range(n):
        for verb in range(n):
            code = memory[:]
            code[1] = noun
            code[2] = verb
            if parse_code(code)[0] == target:
                return noun, verb
    return None
target = 19690720
noun, verb = choose_noun_and_verb(target, memory)
print(100 * noun + verb)

8226


# Day 3: Crossed Wires

In [9]:
with open(input_path(3), 'r') as f:
    paths = [tuple(token for token in line.split(',')) for line in f]
path1, path2 = paths

In [10]:
def compute_wire(path):
    moves = dict(zip("LRUD", ((-1, 0), (1, 0), (0, 1), (0, -1))))
    x, y = 0, 0
    wire = {}
    steps = 0
    for item in path:
        d, cnt = item[0], int(item[1:])
        dx, dy = moves[d]
        for i in range(cnt):
            if (x, y) not in wire:
                wire[x, y] = steps
            x, y = x + dx, y + dy
            steps += 1
    return wire

In [11]:
wire1, wire2 = compute_wire(path1), compute_wire(path2)

In [12]:
print(min(abs(x) + abs(y) for x, y in wire1.keys() & wire2.keys() if x != 0 and y != 0))

232


In [13]:
print(min(wire1[key] + wire2[key] for key in wire1.keys() & wire2.keys() if key != (0, 0)))

6084


# Day 4: Secure Container

In [14]:
lo = 245182
hi = 790572
def valid_passwd(pw):
    pre = 10
    has_double = False
    while pw:
        d = pw % 10
        if pre != 10:
            if pre == d:
                has_double = True
            if pre < d:
                return False
        pre = d
        pw //= 10
    return has_double

In [15]:
print(sum(valid_passwd(pw) for pw in range(lo, hi + 1)))

1099


In [16]:
def valid_passwd(pw):
    pw_str = str(pw)
    pre = None
    has_double = False
    for ch, g in itertools.groupby(pw_str):
        cnt = len(list(g))
        if pre and ch < pre:
            return False
        if cnt == 2:
            has_double = True
        pre = ch
    return has_double

In [17]:
print(sum(valid_passwd(pw) for pw in range(lo, hi + 1)))

710


# Day 5: Sunny with a Chance of Asteroids

In [18]:
with open(input_path(5), 'r') as f:
    code5 = list(map(int, f.read().split(',')))

In [88]:
class Program:
    def __init__(self, code, inputs):
        self.code = code[:]
        self.inputs = inputs
        self.relative_base = 0
        self.genor = self.run()
    
    def run(self):
        code = collections.defaultdict(int)
        for j, c in enumerate(self.code):
            code[j] = c
        n = len(self.code)
        i = 0
        def get(para, mode):
            if mode == 2:
                para += self.relative_base
                mode = 0
            return code[para] if mode == 0 else para
        while True:
            assert(code[i] >= 0)
            instruct = "{:05d}".format(code[i])
            opcode = int(instruct[-2:])
            left_mode = int(instruct[2])
            right_mode = int(instruct[1])
            r_mode = int(instruct[0])
            assert(r_mode == 0)
            if opcode == 99:
                break
            elif opcode == 3:
                code[code[i + 1]] = self.inputs.popleft()
                i += 2
            elif opcode == 4:
                out = get(code[i + 1], left_mode)
                i += 2
#                 print(out)
                yield out
            elif opcode == 1 or opcode == 2:
                op = operator.mul if opcode == 2 else operator.add
                val = op(get(code[i + 1], left_mode), get(code[i + 2], right_mode))
                code[code[i + 3]] = val
                i += 4
            elif opcode == 5:
                first_para = get(code[i + 1], left_mode)
                if first_para != 0:
                    i = get(code[i + 2], right_mode)
                else:
                    i += 3
            elif opcode == 6:
                first_para = get(code[i + 1], left_mode)
                if first_para == 0:
                    i = get(code[i + 2], right_mode)
                else:
                    i += 3
            elif opcode == 7:
                if get(code[i + 1], left_mode) < get(code[i + 2], right_mode):
                    code[code[i + 3]] = 1
                else:
                    code[code[i + 3]] = 0
                i += 4
            elif opcode == 8:
                if get(code[i + 1], left_mode) == get(code[i + 2], right_mode):
                    code[code[i + 3]] = 1
                else:
                    code[code[i + 3]] = 0
                i += 4
            elif opcode == 9:
                self.relative_base += get(code[i + 1], left_mode)
                i += 2
            else:
                assert False, "Unkown opcode {}".format(opcode)

In [89]:
list(Program(code5, deque([1])).genor)

0


[0, 0, 0, 0, 0, 0, 0, 0, 0, 6731945]

In [86]:
test_in = [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
assert(list(Program(test_in, deque([7])).genor)[-1] == 999)

In [87]:
list(Program(code5, deque([5])).genor)[-1]

9571668

# Day 6: Universal Orbit Map

In [23]:
def count_orbits(infile):
    import collections
    tree = collections.defaultdict(list)
    with open(infile, 'r') as f:
        for line in f:
            u, v = line.strip().split(')')
            tree[u].append(v)
    norbits = 0
    def dfs(node, depth):
        nonlocal norbits
        norbits += depth
        for child in tree[node]:
            dfs(child, depth + 1)
    dfs('COM', 0)
    return norbits

In [24]:
# print(count_orbits(input_path_sample(6)))
assert(count_orbits(input_path_sample(6)) == 42)

In [25]:
print(count_orbits(input_path(6)))

301100


In [26]:
def find_path(infile, target="SAN"):
    import collections
    tree = collections.defaultdict(list)
    with open(infile, 'r') as f:
        for line in f:
            u, v = line.strip().split(')')
            tree[u].append(v)
            tree[v].append(u)
    frontier = [("YOU", None, 0)]
    while frontier:
        node, parent, depth = frontier.pop()
        if node == target:
            return depth - 2
        for nei in tree[node]:
            if nei != parent:
                frontier.append((nei, node, depth + 1))

In [27]:
print(find_path(input_path(6)))

547


# Day 7: Amplification Circuit

In [28]:
with open(input_path(7), 'r') as f:
    code7 = list(map(int, f.read().split(',')))

In [29]:
def amplify_circuit(code):
    def amplify(setting):
        signal = 0
        for inst in setting:
            amp = Program(code, deque([inst, signal]))
            signal = next(amp.genor)
        return signal
    return max(amplify(setting) for setting in permutations(range(5)))

In [30]:
print(amplify_circuit(code7))

21000


In [31]:
def feedback_amplify_circuit(code):
    def amplify(setting):
        amps = [Program(code, deque([inst])) for inst in setting]
        ans = None
        signal = 0
        while True:
            for i, amp in enumerate(amps):
                amps[i].inputs.append(signal)
                signal = next(amp.genor, None)
                if signal is None:
                    return ans
                if i == 4:
                    ans = signal
    return max(amplify(setting) for setting in permutations(range(5, 10)))            

In [32]:
assert(feedback_amplify_circuit([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,
                                 1001,28,-1,28,1005,28,6,99,0,0,5]) == 139629729)

In [33]:
assert(feedback_amplify_circuit([3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]) == 18216)

In [34]:
print(feedback_amplify_circuit(code7))

61379886


# Day 8: Space Image Format

In [35]:
WIDTH = 25
HEIGHT = 6
with open(input_path(8), 'r') as f:
    image = f.read().strip()
n = len(image)
assert(n % (WIDTH * HEIGHT) == 0)
assert(image.isdigit())

In [36]:
PIXELS = WIDTH * HEIGHT
counter = [PIXELS + 1] * 10
for i in range(n // (PIXELS)):
    temp = [0] * 10
    start = i * PIXELS
    for i in range(start, start + PIXELS):
        pix = image[i]
        temp[ord(pix) - ord('0')] += 1
    if temp[0] < counter[0]:
        counter = temp
print(counter[1] * counter[2])

1360


In [37]:
def decode_image(image, width=25, height=6):
    n = len(image)
    total = width * height
    ans = [next(itertools.dropwhile(lambda x : x == '2', (image[j] for j in range(i, n, total))) ) for i in range(total)]
    for row in range(height):
        line = ''.join(ans[row * width: row * width + width])
        print(line)
    return ans

In [38]:
decode_image(image);
# FPUAR

1111011100100100110011100
1000010010100101001010010
1110010010100101001010010
1000011100100101111011100
1000010000100101001010100
1000010000011001001010010


# Day 9: Sensor Boost

In [73]:
with open(input_path(9), 'r') as f:
    code9 = list(map(int, f.read().split(',')))

In [74]:
list(Program(code9, deque([1])).genor)

[203, 0]

In [75]:
assert(list(Program([104,1125899906842624,99], deque([1])).genor)[-1] == 1125899906842624)

In [76]:
assert(list(Program([109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99], deque([1])).genor) == [109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99])

In [77]:
assert(len(str(list(Program([1102,34915192,34915192,7,4,7,99,0], deque([1])).genor)[0])) == 16)

# Day 10: Monitoring Station 

In [160]:
with open(input_path_sample(10), 'r') as f:
    grid = [line.strip() for line in f]

In [161]:
print(sum(val == '#' for row in grid for val in row))

300


In [130]:
def best_monitor_station(grid):
    """find the best monitor station."""
    m, n = len(grid), len(grid[0])
    asters = [(x, y) for y in range(m) for x in range(n) if grid[y][x] == '#']
    counter = collections.Counter()
    for i, (x1, y1) in enumerate(asters):
        for j in range(i):
            x0, y0 = asters[j]
            # check if there are any asteroids between (x0, y0) and (x1, y1)
            dx, dy = x1 - x0, y1 - y0
            xmin, xmax = sorted((x0, x1))
            ymin, ymax = sorted((y0, y1))
            for k in range(len(asters)):
                if k not in (i, j):
                    x, y = asters[k]
                    if xmin <= x <= xmax and ymin <= y <= ymax and dy * (x - x0) == dx * (y - y0):
                        break
            else:
                counter[i] += 1
                counter[j] += 1
    best_idx, mx = counter.most_common(1)[0]
    return asters[best_idx], mx

In [133]:
monitor, mx = best_monitor_station(grid)
print("Best location is at ({}, {}) which connects {} asteroid.".format(monitor[0], monitor[1], mx))

Best location is at (11, 13) which connects 210 asteroid


In [156]:

def rotate_and_vaporize(grid, monitor):
    """rotate the laser clockwise and vaporize the asteroids."""
    m, n = len(grid), len(grid[0])
    x0, y0 = monitor
    asters = [(x, y) for y in range(m) for x in range(n) if grid[y][x] == '#' and (x != x0 or y != y0)]
    removes = []
    while asters:
        cands = []
        nas = len(asters)
        next_asters = []
        for i, (x1, y1) in enumerate(asters):
            dx, dy = x1 - x0, y1 - y0
            xmin, xmax = sorted((x0, x1))
            ymin, ymax = sorted((y0, y1))
            for j in range(nas):
                if j != i:
                    x, y = asters[j]
                    if xmin <= x <= xmax and ymin <= y <= ymax and dy * (x - x0) == dx * (y - y0):
                        next_asters.append((x1, y1))
                        break
            else:
                theta = 
                cands.append((np.arctan2(-dy, dx), i))
        cands.sort()
        print([asters[idx] for _, idx in cands])
        removes.extend(asters[idx] for _, idx in cands)
        asters = next_asters
    return removes

In [157]:
rotate_and_vaporize(grid, monitor);

[(0, 14), (2, 14), (3, 14), (4, 14), (5, 14), (6, 14), (2, 15), (7, 14), (4, 15), (8, 14), (6, 15), (4, 16), (0, 18), (9, 14), (0, 19), (2, 18), (1, 19), (8, 15), (4, 18), (7, 16), (6, 17), (4, 19), (10, 14), (7, 18), (8, 17), (7, 19), (8, 18), (10, 15), (9, 18), (10, 16), (10, 17), (10, 18), (10, 19), (11, 14), (12, 19), (12, 18), (12, 17), (12, 16), (13, 18), (12, 15), (14, 18), (15, 19), (14, 17), (15, 18), (13, 15), (18, 19), (17, 18), (15, 16), (18, 18), (14, 15), (19, 18), (16, 16), (18, 17), (13, 14), (18, 16), (16, 15), (19, 16), (14, 14), (18, 15), (15, 14), (16, 14), (17, 14), (18, 14), (19, 14), (12, 13), (19, 12), (18, 12), (17, 12), (15, 12), (14, 12), (19, 10), (16, 11), (18, 10), (13, 12), (16, 10), (19, 8), (17, 9), (18, 8), (15, 10), (16, 9), (17, 8), (18, 7), (19, 6), (12, 12), (19, 4), (18, 5), (17, 6), (16, 7), (15, 8), (18, 4), (14, 9), (19, 2), (16, 6), (18, 3), (13, 10), (16, 5), (19, 0), (14, 8), (18, 1), (15, 6), (16, 4), (17, 2), (18, 0), (14, 7), (17, 0), (16